In [2]:
import pandas as pd

In [3]:
import random
p = 0.004  # 0.4% of the lines
# keep the header, then take only 0.4% of lines
# if random from [0,1] interval is greater than 0.01 the row will be skipped
passenger = pd.read_csv(
         '../data/passengers.csv',
         header=0, 
         skiprows=lambda i: i>0 and random.random() > p
)

In [4]:
passenger.head()

,departures_scheduled,departures_performed,payload,seats,passengers,freight,mail,distance,ramp_to_ramp,air_time,...,dest_country,dest_country_name,aircraft_group,aircraft_type,aircraft_config,year,month,distance_group,class,data_source
0,0.0,1.0,1275.0,5.0,0.0,0.0,0.0,93.0,46.0,42.0,...,US,United States,0,35,1,2015,6,1,L,DU
1,0.0,3.0,16800.0,0.0,0.0,4644.0,0.0,87.0,108.0,90.0,...,US,United States,4,412,2,2015,6,1,G,DU
2,0.0,1.0,18720.0,69.0,69.0,0.0,0.0,391.0,80.0,60.0,...,US,United States,6,677,1,2015,6,1,F,DU
3,0.0,1.0,282198.0,0.0,0.0,0.0,0.0,594.0,115.0,88.0,...,US,United States,8,819,2,2015,6,2,P,DU
4,0.0,1.0,26200.0,100.0,62.0,0.0,0.0,290.0,164.0,80.0,...,US,United States,6,678,1,2015,6,1,F,DU


In [5]:
passenger.to_csv('../data/passenger_reduced.csv')

In [22]:
passengers = passenger.drop(columns=[
    'departures_scheduled', 'departures_performed', 'payload', 'seats',
     'freight', 'mail', 'ramp_to_ramp', 'airline_id', 'unique_carrier_name', 'region',
       'carrier', 'carrier_name', 'carrier_group', 'carrier_group_new',
       'origin_city_market_id', 'origin',
       'origin_city_name', 'origin_country', 'origin_country_name',
       'dest_city_market_id', 'dest', 'dest_city_name',
       'dest_country', 'dest_country_name', 'aircraft_group', 'aircraft_type',
       'aircraft_config', 'year', 'distance_group', 'class',
       'data_source'

])

Passenger Feature Engineering 

In [23]:
passengers

,passengers,distance,air_time,unique_carrier,origin_airport_id,dest_airport_id,month
0,60.0,95.0,85.0,7H,10304,10551,6
1,61.0,1235.0,155.0,CP,12892,11298,6
2,62.0,1363.0,186.0,CP,13198,12892,6
3,0.0,489.0,108.0,GFQ,11526,13198,6
4,134.0,878.0,107.0,WN,14488,13204,6
...,...,...,...,...,...,...,...
9294,96.0,1045.0,137.0,YX,10590,13029,5
9295,3081.0,4677.0,0.0,DE,11760,10299,6
9296,430.0,620.0,3120.0,KS,10917,10299,5
9297,6809.0,5186.0,0.0,AZ,11651,13303,7


In [74]:
# total_pass_per_carrier = passengers.groupby('unique_carrier').agg({'passengers': ['mean', 'sum']})
carrier_pass = passengers.groupby('unique_carrier').passengers.agg(['sum', 'mean'])
carrier_pass.sort_values('sum', ascending=False).reset_index()

,unique_carrier,sum,mean
0,WN,3497049.0,4559.385919
1,DL,3026773.0,3499.159538
2,AA,2756098.0,5210.015123
3,UA,2203029.0,2941.293725
4,B6,729063.0,4072.977654
...,...,...,...
269,U7,0.0,0.000000
270,PFQ,0.0,0.000000
271,ADB,0.0,0.000000
272,V4,0.0,0.000000


In [73]:
month_pass = passengers.groupby('month').passengers.agg(['sum', 'mean'])
month_pass.sort_values('sum', ascending=False).reset_index()

,month,sum,mean
0,12,2042354.0,2490.675610
1,7,1892012.0,2376.899497
2,8,1890055.0,2365.525657
3,5,1823396.0,2386.643979
4,6,1751923.0,2093.097969
5,3,1677675.0,2173.154145
6,4,1545122.0,1955.850633
7,1,1544277.0,2048.112732
8,11,1527114.0,2058.105121
9,10,1460062.0,2019.449516


In [57]:
passengers['speed'] = passengers.distance / passenger.air_time
# miles/minutes 

In [53]:
passengers

,passengers,distance,air_time,unique_carrier,origin_airport_id,dest_airport_id,month,speed
0,60.0,95.0,85.0,7H,10304,10551,6,1.117647
1,61.0,1235.0,155.0,CP,12892,11298,6,7.967742
2,62.0,1363.0,186.0,CP,13198,12892,6,7.327957
3,0.0,489.0,108.0,GFQ,11526,13198,6,4.527778
4,134.0,878.0,107.0,WN,14488,13204,6,8.205607
...,...,...,...,...,...,...,...,...
9294,96.0,1045.0,137.0,YX,10590,13029,5,7.627737
9295,3081.0,4677.0,0.0,DE,11760,10299,6,inf
9296,430.0,620.0,3120.0,KS,10917,10299,5,0.198718
9297,6809.0,5186.0,0.0,AZ,11651,13303,7,inf


--------------------------------------

In [16]:
fuel = pd.read_csv('../data/fuel_consumption .csv')
fuel.head()

,month,airline_id,unique_carrier,carrier,carrier_name,carrier_group_new,sdomt_gallons,satl_gallons,spac_gallons,slat_gallons,...,sdomt_cost,satl_cost,spac_cost,slat_cost,sint_cost,ts_cost,tdomt_cost,tint_cost,total_cost,year
0,1,NaN,NaN,0JQ,NaN,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5100.0,0.0,5100.0,2016
1,1,21352.0,0WQ,0WQ,Avjet Corporation,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,396216.0,140239.0,536455.0,2016
2,1,21645.0,23Q,23Q,Songbird Airways Inc.,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016
3,1,21652.0,27Q,27Q,"Jet Aviation Flight Services, Inc.",1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016
4,1,20408.0,5V,5V,Tatonduk Outfitters Limited d/b/a Everts Air A...,1,260848.0,0.0,0.0,0.0,...,522405.0,0.0,0.0,0.0,0.0,522405.0,569497.0,0.0,569497.0,2016


In [17]:
fuel.shape

(3035, 25)

In [24]:
fuel.columns

Index(['month', 'airline_id', 'unique_carrier', 'carrier', 'carrier_name',
       'carrier_group_new', 'sdomt_gallons', 'satl_gallons', 'spac_gallons',
       'slat_gallons', 'sint_gallons', 'ts_gallons', 'tdomt_gallons',
       'tint_gallons', 'total_gallons', 'sdomt_cost', 'satl_cost', 'spac_cost',
       'slat_cost', 'sint_cost', 'ts_cost', 'tdomt_cost', 'tint_cost',
       'total_cost', 'year'],
      dtype='object')

In [26]:
fuel_con = fuel.drop(columns=['airline_id','carrier', 'carrier_name',
       'carrier_group_new', 'sdomt_gallons', 'satl_gallons', 'spac_gallons',
       'slat_gallons', 'sint_gallons', 'ts_gallons', 'tdomt_gallons',
       'tint_gallons','sdomt_cost', 'satl_cost', 'spac_cost',
       'slat_cost', 'sint_cost', 'ts_cost', 'tdomt_cost', 'tint_cost','year'
       ])

Fuel Consumption Feature Engineering 

In [27]:
fuel_con

,month,unique_carrier,total_gallons,total_cost
0,1,NaN,3000.0,5100.0
1,1,0WQ,210112.0,536455.0
2,1,23Q,0.0,0.0
3,1,27Q,0.0,0.0
4,1,5V,284362.0,569497.0
...,...,...,...,...
3030,12,X9,2531738.0,6172701.0
3031,12,XP,0.0,0.0
3032,12,YV,0.0,0.0
3033,12,YX,21048.0,50043.0


In [72]:
fuel_per_month = fuel_con.groupby('month').total_gallons.agg(['sum', 'mean'])
fuel_per_month.sort_values('sum',ascending=False).reset_index()

,month,sum,mean
0,7,8.390187e+09,3.142392e+07
1,8,8.258306e+09,3.116342e+07
2,6,8.057899e+09,3.052234e+07
3,5,7.816662e+09,2.927589e+07
4,3,7.601791e+09,2.836489e+07
5,4,7.417560e+09,2.788557e+07
6,9,7.386270e+09,2.776793e+07
7,1,7.044165e+09,2.648182e+07
8,2,6.445816e+09,2.414163e+07
9,12,6.052621e+09,2.882200e+07


In [71]:
fuel_cost_per_month = fuel_con.groupby('month').total_cost.agg(['sum', 'mean'])
fuel_cost_per_month.sort_values('sum',ascending=False).reset_index()

,month,sum,mean
0,6,1.561230e+10,5.913751e+07
1,7,1.533858e+10,5.744786e+07
2,8,1.484116e+10,5.600439e+07
3,5,1.482041e+10,5.550717e+07
4,3,1.368703e+10,5.107101e+07
5,4,1.356377e+10,5.080064e+07
6,9,1.346830e+10,5.044306e+07
7,1,1.252890e+10,4.692472e+07
8,2,1.194639e+10,4.474302e+07
9,10,1.122771e+10,5.321190e+07


In [70]:
fuel_cost_per_carrier = fuel_con.groupby('unique_carrier').total_cost.agg(['sum', 'mean'])
fuel_cost_per_carrier.sort_values('sum',ascending=False).reset_index()

,unique_carrier,sum,mean
0,DL,3.286616e+10,5.765994e+08
1,AA,2.853022e+10,5.005301e+08
2,UA,2.786589e+10,4.888753e+08
3,WN,1.816020e+10,3.186000e+08
4,FX,9.372142e+09,1.644235e+08
...,...,...,...
57,9E,0.000000e+00,0.000000e+00
58,OH,0.000000e+00,0.000000e+00
59,CP,0.000000e+00,0.000000e+00
60,MQ,0.000000e+00,0.000000e+00


In [69]:
fuel_per_carrier = fuel_con.groupby('unique_carrier').total_gallons.agg(['sum', 'mean'])
fuel_per_carrier.sort_values('sum',ascending=False).reset_index()

,unique_carrier,sum,mean
0,AA,1.663489e+10,2.918401e+08
1,DL,1.655025e+10,2.903553e+08
2,UA,1.595691e+10,2.799457e+08
3,WN,9.599040e+09,1.684042e+08
4,FX,5.464666e+09,9.587134e+07
...,...,...,...
57,9E,0.000000e+00,0.000000e+00
58,OH,0.000000e+00,0.000000e+00
59,CP,0.000000e+00,0.000000e+00
60,MQ,0.000000e+00,0.000000e+00
